In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pylab as plt

2023-04-11 10:54:30.465974: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 10:54:30.619793: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-04-11 10:54:30.621105: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-11 10:54:31.486752: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
attribute_path = "/home/ali/projects/photo_collection/celebface-archive/list_attr_celeba.csv"
partition_path = "/home/ali/projects/photo_collection/celebface-archive/list_eval_partition.csv"
images_path = "/home/ali/projects/photo_collection/celebface-archive/img_align_celeba/img_align_celeba/"
batch_size = 128
Nodel_handle = "https://tfhub.dev/google/imagenet/efficientnet_v2_imagenet21k_ft1k_b0/feature_vector/2"
image_size = (224, 224)

In [8]:
df_attributes = pd.read_csv(attribute_path)

In [9]:
df_attributes.columns

Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

In [57]:
selected_attributes = ["5_o_Clock_Shadow", "Bald", "Bangs", "Black_Hair", "Blond_Hair", "Brown_Hair",
                       "Eyeglasses", "Goatee", "Gray_Hair", "Male", "Mustache", "No_Beard", "Sideburns", "Smiling", "Straight_Hair", "Wavy_Hair",
                       "Wearing_Earrings", "Wearing_Hat", "Wearing_Lipstick", "Wearing_Necklace", "Wearing_Necktie"]

In [60]:
column_list = ["image_id"]
column_list.extend(selected_attributes)

In [62]:
df_attributes_selected = df_attributes[df_attributes.columns.intersection(column_list)].copy()

In [63]:
df_attributes_selected.head()

,image_id,5_o_Clock_Shadow,Bald,Bangs,Black_Hair,Blond_Hair,Brown_Hair,Eyeglasses,Goatee,Gray_Hair,...,No_Beard,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie
0,000001.jpg,-1,-1,-1,-1,-1,1,-1,-1,-1,...,1,-1,1,1,-1,1,-1,1,-1,-1
1,000002.jpg,-1,-1,-1,-1,-1,1,-1,-1,-1,...,1,-1,1,-1,-1,-1,-1,-1,-1,-1
2,000003.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,1,-1,-1,-1,-1,-1
3,000004.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,1,-1,1,-1,1,1,-1
4,000005.jpg,-1,-1,-1,-1,-1,-1,-1,-1,-1,...,1,-1,-1,-1,-1,-1,-1,1,-1,-1


In [69]:
df_partition = pd.read_csv(partition_path)

In [70]:
df = pd.merge(df_attributes_selected, df_partition, on="image_id")
df = df.replace(-1,0)
df.head()

,image_id,5_o_Clock_Shadow,Bald,Bangs,Black_Hair,Blond_Hair,Brown_Hair,Eyeglasses,Goatee,Gray_Hair,...,Sideburns,Smiling,Straight_Hair,Wavy_Hair,Wearing_Earrings,Wearing_Hat,Wearing_Lipstick,Wearing_Necklace,Wearing_Necktie,partition
0,000001.jpg,0,0,0,0,0,1,0,0,0,...,0,1,1,0,1,0,1,0,0,0
1,000002.jpg,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,000003.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,000004.jpg,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,1,1,0,0
4,000005.jpg,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [112]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, image_size)
    return image

In [421]:
#TODO is there a better way to do this?

def load_and_preprocess_image(path, *args):
    image = tf.io.read_file(path)
    return preprocess_image(image), *args

In [ ]:
# def build_dataset_from_df(input_df, attr_list):
#     a_list = [images_path + image_id for image_id in input_df["image_id"]]
#     a_list.append([input_df[attr] for attr in attr_list])
#     a_tuple = tuple(a_list)
#     dataset = tf.data.Dataset.from_tensor_slices(a_tuple)
#     # dataset = dataset.map()
#     # dataset = dataset.shuffle(buffer_size=1000)
#     # dataset = dataset.repeat()
#     # dataset = dataset.batch(batch_size=)
#     # dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
#     return dataset

In [422]:
def build_dataset_from_df(input_df, attr_list):
    b_list = [input_df[attr] for attr in attr_list]
    b_list.insert(0, [images_path + image_id for image_id in input_df["image_id"]])
    a_tuple = tuple(b_list)
    dataset = tf.data.Dataset.from_tensor_slices(a_tuple)
    dataset = dataset.map(load_and_preprocess_image)

    return dataset

In [423]:
ds = build_dataset_from_df(df, selected_attributes)

In [424]:
list(ds.take(3).as_numpy_iterator())

2023-04-11 17:48:06.469524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [202599]
	 [[{{node Placeholder/_0}}]]


[(array([[[251.      , 232.      , 192.      ],
          [251.      , 232.      , 192.      ],
          [251.      , 232.      , 192.      ],
          ...,
          [249.62057 , 232.13397 , 218.08038 ],
          [253.69196 , 237.30804 , 220.38391 ],
          [253.      , 238.      , 219.      ]],
  
         [[251.      , 232.      , 192.      ],
          [251.      , 232.      , 192.      ],
          [251.      , 232.      , 192.      ],
          ...,
          [250.55467 , 233.56085 , 219.50726 ],
          [253.98763 , 238.26787 , 221.34373 ],
          [253.95982 , 238.95982 , 219.95982 ]],
  
         [[251.      , 232.      , 192.      ],
          [251.      , 232.      , 192.      ],
          [251.      , 232.      , 192.      ],
          ...,
          [251.50182 , 234.52861 , 220.47502 ],
          [254.      , 238.30804 , 221.38391 ],
          [254.      , 239.      , 220.      ]],
  
         ...,
  
         [[139.19653 ,  73.19653 ,  23.330444],
          [122

In [328]:
selected_attributes

['5_o_Clock_Shadow',
 'Bald',
 'Bangs',
 'Black_Hair',
 'Blond_Hair',
 'Brown_Hair',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Male',
 'Mustache',
 'No_Beard',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie']

In [330]:
tuple([attr for attr in selected_attributes])

('5_o_Clock_Shadow',
 'Bald',
 'Bangs',
 'Black_Hair',
 'Blond_Hair',
 'Brown_Hair',
 'Eyeglasses',
 'Goatee',
 'Gray_Hair',
 'Male',
 'Mustache',
 'No_Beard',
 'Sideburns',
 'Smiling',
 'Straight_Hair',
 'Wavy_Hair',
 'Wearing_Earrings',
 'Wearing_Hat',
 'Wearing_Lipstick',
 'Wearing_Necklace',
 'Wearing_Necktie')